In [1]:
#Defines directories and credent
execfile('theStuff.py')

import psycopg2
import getpass
import psycopg2.extras
import pandas as pd
import numpy as np

In [2]:

conn_string = "host='localhost' dbname=" + str(database) + " user=" + str(pgu) + " password=" + str(passpg)

conn= psycopg2.connect(conn_string)


In [3]:
# Identifies proper nouns in sentences containing word dam.  This can be used to query dam names, stream names, states and alike
# that may help to identify main dam removal(s) referenced in a paper.

def properNouns(cursor):
    d = []
    #Requires cursor from postgres query of initial NLP data from UW database
    for sentence in cursor:
        #Identify fields within cursor
        docid = sentence[0]
        sentid = sentence[1]
        words = sentence[3]
        poses = sentence[4]
        index_num = len(poses)   # Set index for total number of items within sentence
        
        ####################################################################################################
        ###Loop through each sentence item################################################################## 
        #if Proper Noun return docid, sentid, begin_index, end_index, and entire string of Proper Noun Words
        ###################################################################################################
        
        for i in xrange(index_num): 
            #try used to catch first iteration where end_index is not currently defined
            try:   
                # If catches all Proper Nouns that are more than one word, elif catches Proper Nouns = 1 word
                # if and elif require end index to be less than index so partial string of Proper Nouns are not captured 
                # where end_index is greater than index continue to next index value in sentence 
                if i > end_index and poses[i] == "NNP" and poses[i+1] == 'NNP':
                    end_index = i + 1
                    proper_noun = words[i]
                    while end_index < index_num and (poses[end_index] == "NNP"):
                        proper_noun += (" " + words[end_index])
                        mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                        end_index += 1
                elif i > end_index and poses[i] == "NNP":
                    end_index = i 
                    mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                    proper_noun = words[i]
                else:
                    continue
            except:
                if poses[i] == "NNP" and poses[i+1] == 'NNP':
                    end_index = i + 1
                    proper_noun = words[i]
                    while end_index < index_num and (poses[end_index] == "NNP"):
                        proper_noun += (" " + words[end_index])
                        mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                        end_index += 1
                elif poses[i] == "NNP":
                    end_index = i 
                    mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                    proper_noun = words[i]
                else: 
                    continue
            d.append({"mention_id": mention_id, 'docid': docid, 'sentid': sentid, 'begin_index': i, 'end_index': end_index, 'proper_noun': proper_noun})


            
        #Clear variables: mostly for rerunning in ipython notebook
        try:
            del end_index
        except:
            continue
        try:
            del i
        except:
            continue
    return d

In [4]:

import pandas as pd
#returns sentences with work dam in it
#cursor.execute("select b.* from (select docid, sentid, unnest(words) as word from sentences) a left join sentences b on (a.docid=b.docid AND a.sentid=b.sentid) where (lower(a.word) = 'dam' OR lower(a.word) LIKE '%-dam' OR lower(a.word) LIKE 'dam-%') OR lower(a.word) = 'dams' ;")

#Small query return for testing purposes
cursor = conn.cursor()
cursor.execute("select * from public.sentences where docid = '54b43277e138239d86852119' and (sentid in (42, 43, 44, 45, 46, 47, 545));")
data = properNouns(cursor)
cursor.close()
      
df=pd.DataFrame(data)
df.head(15)


            

,begin_index,docid,end_index,mention_id,proper_noun,sentid
0,0,54b43277e138239d86852119,0,54b43277e138239d86852119_43_0_0,Exhumation,43
1,13,54b43277e138239d86852119,13,54b43277e138239d86852119_43_13_13,Brandon,43
2,18,54b43277e138239d86852119,18,54b43277e138239d86852119_44_18_18,Strait,44
3,20,54b43277e138239d86852119,20,54b43277e138239d86852119_44_20_20,Juan,44
4,22,54b43277e138239d86852119,22,54b43277e138239d86852119_44_22_22,Fuca,44
5,24,54b43277e138239d86852119,24,54b43277e138239d86852119_44_24_24,Polenz,44
6,8,54b43277e138239d86852119,10,54b43277e138239d86852119_46_8_9,Elwha River,46
7,7,54b43277e138239d86852119,9,54b43277e138239d86852119_545_7_8,Elwha River,545
8,10,54b43277e138239d86852119,10,54b43277e138239d86852119_545_10_10,Washington,545
